## About HAPI
The Heliophysics Data Application Programmer’s Interface (HAPI) specification is a time series download and streaming format specification. A 1-page summary is given in HAPI_OnePager.pdf. A more detailed overview is given in the JGR article Weigel et al., 2021 and the presentation by Vandegriff et al. (pdf|YouTube video).

The HAPI specification was recommended by COSPAR in 2018 as the common data access API for space science and space weather data.

When data are available from a HAPI server, there is no need to download data files and write custom file reader programs. Using a HAPI client library, data can be loaded into an array using a single command using HAPI IDL, MATLAB, and Python clients. Data from HAPI servers is also accessible to users of Autoplot, PySPEDAS, and IDL SPEDAS. Sample scripts and containing instructions for accessing data using the above–listed clients may be found by selecting a server, dataset, and time range at the web interface http://hapi-server.org/servers/.

A list of HAPI-compliant data servers is available at http://hapi-server.org/servers/.

The hapi-server GitHub project contains a collection of repositories for HAPI–related software and documentation, including client and server libraries and code for verifying and testing a HAPI server.

Go to the server [here](https://hapi-server.org/)

In [ ]:
# Python 2.7 and 3 compatible
# Report software bugs/issues/feature requests at
# https://github.com/hapi-server/client-python/issues
# Report data server issues to NASA-SPDF-Support@nasa.onmicrosoft.com

# Install latest hapiclient package from https://pypi.org/project/hapiclient/
# Only needs to be executed once.
import os; print(os.popen('pip install hapiclient --upgrade').read())

from hapiclient import hapi

server     = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset    = 'PSP_HELIO1DAY_POSITION'
# Notes:
# 1. Use parameters='' to request all parameters from PSP_HELIO1DAY_POSITION.
# 2. Multiple parameters can be requested using a comma-separated
#    list, e.g., parameters='RAD_AU,SE_LAT'
parameters = ''
start      = '2018-08-13T00:00:00Z' # min 2018-08-13T00:00:00Z
stop       = '2029-08-15T00:00:00.000Z' # max 2029-01-31T00:00:00Z

data, meta = hapi(server, dataset, parameters, start, stop)

import os; print(os.popen('pip install hapiplot --upgrade').read())
from hapiplot import hapiplot
hapiplot(data, meta)

# Notes:
# 1. To convert ISO 8601 strings the primary time parameter to Python
#    datetimes, use
#      from hapiclient import hapitime2datetime
#      time_name = meta["parameters"][0]["name"] # Primary time parameter is always first.
#      Time = hapitime2datetime(data[time_name])
# 2. Details about the data and metadata structures `data`
#    and `meta` are given at 
#    https://github.com/hapi-server/client-python-notebooks/blob/master/hapi_demo.ipynb
# 3. Many examples for using `data` and `meta` with other
#    Python libraries (e.g., Pandas, Numpy, Astropy) are given
#    in above-referenced notebook.

In [ ]:
#df = pd.DataFrame(array, columns=['Time', 'RadAU', 'SE_LAT','SE_LON','HG_LAT','HG_LON','HGI_LAT','HGI_LON']) 
from hapiclient import hapi

server     = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset    = 'PSP_HELIO1DAY_POSITION'
# Notes:
# 1. Use parameters='' to request all parameters from PSP_HELIO1DAY_POSITION.
# 2. Multiple parameters can be requested using a comma-separated
#    list, e.g., parameters='RAD_AU,SE_LAT'
parameters = ''
start      = '2018-08-13T00:00:00Z' # min 2018-08-13T00:00:00Z
stop       = '2029-08-15T00:00:00.000Z' # max 2029-01-31T00:00:00Z

data, meta = hapi(server, dataset, parameters, start, stop)


In [ ]:
import numpy as np
import pandas as pd
data_dict = {name: data[name] for name in data.dtype.names}
# Convert to DataFrame
df = pd.DataFrame([data_dict])
df['Time'] = df['Time'].apply(lambda x: x[0].decode('utf-8') if isinstance(x, (list, np.ndarray)) else x)
# Convert to datetime64
df['Time'] = pd.to_datetime(df['Time'])


In [ ]:
# Convert the column into multiple rows
df_exploded = df.explode('RAD_AU', ignore_index=True)
#df_exploded

In [ ]:
df_exploded.to_csv("./data/psp_1day_output.csv", index=False)

# VENUS

In [ ]:
from astropy.coordinates import get_body_barycentric
from astropy.time import Time
import pandas as pd

# Generate dates from 2018 to 2025 in 1-day increments
dates = pd.date_range(start="2018-01-01", end="2025-12-31", freq="D")
time = Time(dates)

planets = ['venus', 'mercury', 'earth']

for pp in planets:
    # Get plane position relative to the Sun (barycentric coordinates)
    positions = get_body_barycentric(pp, time)
    
    # Convert to DataFrame
    df = pd.DataFrame({
        "date": dates,
        "x": positions.x.to_value("AU"),
        "y": positions.y.to_value("AU"),
        "z": positions.z.to_value("AU"),
    })
    
    # Print sample data
    print(df.head())
    df.to_csv("./data/"+pp+"_1day_output.csv", index=False)

In [ ]:
pip install spiceypy

In [20]:
import spiceypy as spice

spice.kclear()  # Clear any previously loaded kernels

# Load the kernels
spice.furnsh("spice_kernels/naif0012.tls")  # Leap seconds
spice.furnsh("spice_kernels/de440s.bsp")    # Planetary ephemeris
spice.furnsh("spice_kernels/spp_nom_20180812_20300101_v042_PostV7.bsp")  # PSP nominal trajectory

# List loaded kernels
for i in range(spice.ktotal("ALL")):
    print(spice.kdata(i, "ALL", 100, 100, 100))


('spice_kernels/naif0012.tls', 'TEXT', '', 0)
('spice_kernels/de440s.bsp', 'SPK', '', 7)
('spice_kernels/spp_nom_20180812_20300101_v042_PostV7.bsp', 'SPK', '', 8)


In [21]:
# List available objects in the PSP kernel
ids = spice.spkobj("spice_kernels/spp_nom_20180812_20300101_v042_PostV7.bsp")
print("Available SPICE IDs:", ids)

# Print names associated with each ID
for obj_id in ids:
    try:
        print(f"Object ID {obj_id}: {spice.bodc2n(obj_id)}")
    except spice.SpiceyError:
        print(f"Object ID {obj_id}: (No name found)")


Available SPICE IDs: <SpiceCell dtype = 2, length = 0, size = 1000, card = 1, is_set = 1, adjust = 0, init = 1, base = 1302132131104, data = 1302132131128>
Object ID -96: SOLAR PROBE PLUS


In [22]:
PSP_ID = "-96: SOLAR PROBE PLUS"  # Replace with actual ID found

positions = [spice.spkpos(PSP_ID, et, "ECLIPJ2000", "NONE", "SUN")[0] for et in et_times]


SpiceIDCODENOTFOUND: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(IDCODENOTFOUND) --

The target, '-96: SOLAR PROBE PLUS', is not a recognized name for an ephemeris object. The cause of this problem may be that you need an updated version of the SPICE toolkit. Alternatively you may call SPKEZP directly if you know the SPICE id-codes for both '-96: SOLAR PROBE PLUS' and 'SUN'

spkpos_c --> SPKPOS

================================================================================

In [19]:
from datetime import datetime, timedelta
import pandas as pd

# Generate date range (1-day increments)
start_date = datetime(2018, 1, 1)
end_date = datetime(2025, 12, 31)
dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Convert dates to Ephemeris Time (ET)
et_times = [spice.str2et(date.strftime("%Y-%m-%d")) for date in dates]

# ✅ Use the correct NAIF ID found in Step 2
PSP_ID = "-96: SOLAR PROBE PLUS"  # Update this if a different ID is found

# Compute PSP's position relative to the Sun
positions = [spice.spkpos(PSP_ID, et, "ECLIPJ2000", "NONE", "SUN")[0] for et in et_times]

# Convert to DataFrame
df = pd.DataFrame({
    "date": dates,
    "x": [pos[0] / 1.496e+8 for pos in positions],  # Convert km to AU
    "y": [pos[1] / 1.496e+8 for pos in positions],
    "z": [pos[2] / 1.496e+8 for pos in positions],
})

print(df.head())

# Unload SPICE kernels
spice.kclear()


SpiceIDCODENOTFOUND: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(IDCODENOTFOUND) --

The target, '-96: SOLAR PROBE PLUS', is not a recognized name for an ephemeris object. The cause of this problem may be that you need an updated version of the SPICE toolkit. Alternatively you may call SPKEZP directly if you know the SPICE id-codes for both '-96: SOLAR PROBE PLUS' and 'SUN'

spkpos_c --> SPKPOS

================================================================================

In [9]:
import spiceypy as spice
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Load SPICE kernels (Update the paths if necessary)
spice.furnsh("spice_kernels/naif0012.tls")  # Leap seconds
spice.furnsh("spice_kernels/de440s.bsp")    # Planetary ephemeris
spice.furnsh("spice_kernels/psp_rec.bsp")   # Parker Solar Probe trajectory

# Generate date range (1-day increments)
start_date = datetime(2018, 1, 1)
end_date = datetime(2025, 12, 31)
dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Convert dates to Ephemeris Time (ET)
et_times = [spice.str2et(date.strftime("%Y-%m-%d")) for date in dates]

# Compute PSP’s position relative to the Sun
positions = [spice.spkpos("PSP", et, "ECLIPJ2000", "NONE", "SUN")[0] for et in et_times]

# Convert to Pandas DataFrame
df = pd.DataFrame({
    "date": dates,
    "x": [pos[0] / 1.496e+8 for pos in positions],  # Convert km to AU
    "y": [pos[1] / 1.496e+8 for pos in positions],
    "z": [pos[2] / 1.496e+8 for pos in positions],
})

# Print sample data
print(df.head())

# Unload SPICE kernels
spice.kclear()


SpiceNOSUCHFILE: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(NOSUCHFILE) --

The attempt to load "spice_kernels/psp_rec.bsp" by the routine FURNSH failed. It could not be located.

furnsh_c --> FURNSH --> ZZLDKER

================================================================================

In [10]:
import spiceypy as spice
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Load the PSP SPICE kernel (update the path)
spice.furnsh("spice_kernels/psp_pred.bsp")

# Generate date range from 2018 to 2025 in 1-day increments
start_date = datetime(2018, 1, 1)
end_date = datetime(2025, 12, 31)
dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Convert dates to ET (Ephemeris Time)
et_times = [spice.str2et(date.strftime("%Y-%m-%d")) for date in dates]

# Compute PSP's position relative to the Sun
positions = [spice.spkpos("PSP", et, "ECLIPJ2000", "NONE", "SUN")[0] for et in et_times]

# Convert to DataFrame
df = pd.DataFrame({
    "date": dates,
    "x": [pos[0] / 1.496e+8 for pos in positions],  # Convert km to AU
    "y": [pos[1] / 1.496e+8 for pos in positions],
    "z": [pos[2] / 1.496e+8 for pos in positions],
})

# Print sample data
print(df.head())

# Unload SPICE kernels
spice.kclear()


SpiceIDCODENOTFOUND: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(IDCODENOTFOUND) --

The target, 'PSP', is not a recognized name for an ephemeris object. The cause of this problem may be that you need an updated version of the SPICE toolkit. Alternatively you may call SPKEZP directly if you know the SPICE id-codes for both 'PSP' and 'SUN'

spkpos_c --> SPKPOS

================================================================================

In [6]:
import spiceypy as spice
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Load the necessary SPICE kernels
spice.furnsh("spice_kernels/naif0012.tls")  # Leap seconds kernel
spice.furnsh("spice_kernels/de440s.bsp")    # Planetary ephemeris kernel
spice.furnsh("spice_kernels/psp_pred.bsp")   # Parker Solar Probe trajectory

# Generate dates from 2018 to 2025 in 1-day increments
start_date = datetime(2018, 1, 1)
end_date = datetime(2025, 12, 31)
dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Convert dates to Ephemeris Time (ET)
et_times = [spice.str2et(date.strftime("%Y-%m-%d")) for date in dates]

# Compute PSP’s position relative to the Sun
positions = [spice.spkpos("PSP", et, "ECLIPJ2000", "NONE", "SUN")[0] for et in et_times]

# Convert to Pandas DataFrame
df = pd.DataFrame({
    "date": dates,
    "x": [pos[0] / 1.496e+8 for pos in positions],  # Convert km to AU
    "y": [pos[1] / 1.496e+8 for pos in positions],
    "z": [pos[2] / 1.496e+8 for pos in positions],
})

# Print sample data
print(df.head())

# Unload SPICE kernels
spice.kclear()


SpiceIDCODENOTFOUND: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(IDCODENOTFOUND) --

The target, 'PSP', is not a recognized name for an ephemeris object. The cause of this problem may be that you need an updated version of the SPICE toolkit. Alternatively you may call SPKEZP directly if you know the SPICE id-codes for both 'PSP' and 'SUN'

spkpos_c --> SPKPOS

================================================================================

In [11]:
import spiceypy as spice

spice.kclear()  # Clear all kernels
spice.furnsh("spice_kernels/naif0012.tls")  # Leap seconds
spice.furnsh("spice_kernels/de440s.bsp")    # Planetary ephemeris
spice.furnsh("spice_kernels/psp_pred.bsp")  # PSP kernel (update filename if needed)

# Print loaded kernels
print(spice.kdata(0, "SPK", 100, 100, 100))

('spice_kernels/de440s.bsp', 'SPK', '', 3)


In [12]:
# Get a list of all available objects in the loaded kernels
ids = spice.spkobj("spice_kernels/psp_pred.bsp")
print("Available SPICE IDs:", ids)


Available SPICE IDs: <SpiceCell dtype = 2, length = 0, size = 1000, card = 1, is_set = 1, adjust = 0, init = 1, base = 1302101016368, data = 1302101016392>


In [13]:
for obj_id in ids:
    print(f"Object ID {obj_id}: {spice.bodc2n(obj_id)}")  # Get object name

Object ID -96: SOLAR PROBE PLUS


In [15]:
positions = [spice.spkpos("-96: SOLAR PROBE PLUS", et, "ECLIPJ2000", "NONE", "SUN")[0] for et in et_times]

SpiceIDCODENOTFOUND: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(IDCODENOTFOUND) --

The target, '-96: SOLAR PROBE PLUS', is not a recognized name for an ephemeris object. The cause of this problem may be that you need an updated version of the SPICE toolkit. Alternatively you may call SPKEZP directly if you know the SPICE id-codes for both '-96: SOLAR PROBE PLUS' and 'SUN'

spkpos_c --> SPKPOS

================================================================================

In [ ]:
from astroquery.jplhorizons import Horizons
import pandas as pd
import time  # To prevent API rate limits

# Generate date range from 2018 to 2025 in 1-day increments
dates = pd.date_range(start="2018-01-01", end="2025-12-31", freq="D")

# Empty list to store results
data = []

# Fetch data in chunks to avoid API errors
for date in dates:
    date_str = date.strftime("%Y-%m-%d")  # Convert to JPL Horizons format

    try:
        # Query PSP (-96) relative to the Sun (500@10)
        obj = Horizons(id='-96', location='500@10', epochs=[date_str]).vectors()

        # Append results
        data.append({
            "date": pd.to_datetime(obj['datetime_jd'], origin='julian', unit='D'),
            "x": float(obj['x']),
            "y": float(obj['y']),
            "z": float(obj['z'])
        })

        time.sleep(0.5)  # Prevent overwhelming the server

    except Exception as e:
        print(f"Error fetching data for {date_str}: {e}")

# Convert to DataFrame
df = pd.DataFrame(data)

# Print sample data
print(df.head())


df.to_csv("./data/psp_1day_output_same_planet_format.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt

server     = 'https://amda.irap.omp.eu/service/hapi'
dataset    = 'psp-orb-all'

parameters = ''
start      = '2018-08-12T08:16:23Z' # min 2018-08-12T08:16:23Z
stop       = '2025-02-05T08:16:23.000Z' # max 2025-08-30T23:00:00Z

data, meta = hapi(server, dataset, parameters, start, stop)

# 1. To convert ISO 8601 strings the primary time parameter to Python
#    datetimes, use
#      from hapiclient import hapitime2datetime
#      time_name = meta["parameters"][0]["name"] # Primary time parameter is always first.
#      Time = hapitime2datetime(data[time_name])
# 2. Details about the data and metadata structures `data`
#    and `meta` are given at 
#    https://github.com/hapi-server/client-python-notebooks/blob/master/hapi_demo.ipynb
# 3. Many examples for using `data` and `meta` with other
#    Python libraries (e.g., Pandas, Numpy, Astropy) are given
#    in above-referenced notebook.

In [ ]:
data_dict = {name: data[name] for name in data.dtype.names}
# Convert to DataFrame
df = pd.DataFrame([data_dict])
df['Time'] = df['Time'].apply(lambda x: x[0].decode('utf-8') if isinstance(x, (list, np.ndarray)) else x)
# Convert to datetime64
df['Time'] = pd.to_datetime(df['Time'])
#df.to_csv("./data/psp_venus_output.csv", index=False)

In [ ]:
df

In [ ]:
df.psp_lon_sun[0].size

In [ ]:
lon = np.linspace(0, 360, 100)  # Longitude from 0 to 360 degrees
lon = df.psp_lon_sun[0][0:3000]
radius = df.psp_r_sun[0][0:3000]  # Varying radius for demonstration

# Convert Longitude to Radians
theta = np.radians(lon)

# Create Polar Plot
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

# Plot the Object's Path
ax.plot(theta, radius, label="Orbiting Object")

# Mark the Center (Central Body)
ax.plot(0, 0, 'ro', markersize=10, label="Central Object")

# Formatting
ax.set_theta_zero_location('E')  # Set 0° at the right (East)
ax.set_theta_direction(-1)       # Clockwise direction
ax.set_rticks([])                # Remove radial ticks
ax.legend()


In [ ]:
plt.plot(df.psp_lon_sun[0],df.psp_r_sun[0])
#plt.plot(df.psp_lon_hci[0],df.psp_lat_hci[0])
#plt.plot(df.psp_lon_hee[0],df.psp_lat_hee[0])

In [ ]:
print(df.dtypes)  # Check column types
print(df.head())  # Inspect the first few rows
#df2=df
for col in df.columns:
    if col=='Time':
        continue
    #df[~df[col].apply(lambda x: isinstance(x, (int, float)))]
    #df[col] = df[col].astype(str).str.extract(r'(\d+\.?\d*)')
    df[col] = pd.to_numeric(df[col], errors='ignore
#    df2[col] = pd.to_numeric(df[col])
#print(df2.dtypes)  # Check column types
#print(df2.head())  # Inspect the first few rows


In [ ]:
print(df.dtypes)  # Check column types
print(df.head())  # Inspect the first few rows



In [ ]:
df.psp_lon_sun[0:10]

In [ ]:
df